In [344]:
np.random.seed(42)

- Yêu cầu: Implement Batch Gradient Descent with early stopping for Softmax Regression without using Scikit-Learn (trừ train_test_split)
- Data: iris.csv

# Import

In [345]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Load dataset

In [346]:
iris = datasets.load_iris()
list(iris.keys())

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename']

In [347]:
iris['feature_names']# coi thứ tự 

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

Nhận xét: có 4 cột + 1 cột class

# Chia tập dữ liệu thành 3 tập train/test/valid

## Chuẩn bị X_df và y_sr
- Trong lab này chỉ lấy 2 thuộc tính: petal length, petal width

In [348]:
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]
X[:5]

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2]])

- Thêm cột bias (i.e. $x_0$ = 1) cho X vào lưu vào `X_b` (X with bias). Lưu ý về mặt kí hiệu của x: lowerscript = cột thứ mấy; superscript = hàng thứ mấy

In [349]:
X_b = np.concatenate((np.ones((len(X), 1)), X), axis=1)
X_b[:5]

array([[1. , 1.4, 0.2],
       [1. , 1.4, 0.2],
       [1. , 1.3, 0.2],
       [1. , 1.5, 0.2],
       [1. , 1.4, 0.2]])

- Chia tập

In [350]:
X_train, X_test, y_train, y_test = train_test_split(X_b, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(len(X_train), len(X_val), len(X_test))
y_train

96 24 30


array([2, 1, 1, 0, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 1, 0, 0, 1, 2, 0, 0, 2,
       2, 1, 2, 1, 1, 0, 2, 0, 1, 2, 0, 2, 0, 0, 2, 0, 2, 1, 2, 0, 0, 0,
       0, 2, 0, 0, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 0, 0, 2, 0,
       0, 0, 2, 1, 2, 1, 2, 1, 1, 0, 0, 2, 1, 0, 2, 1, 2, 1, 2, 1, 1, 2,
       1, 1, 0, 2, 0, 1, 0, 0])

Nhận xét: `y_train` đã mã hóa sẵn 3 loài hoa thành 3 số khác nhau. Tuy nhiên để xài dc softmax thì `y_train` phải là matrix mx3 (m = len(X_train)) là để xíu xài cho gradient có đụng. Xét 1 hàng/instance x, với mỗi cột k (trong 3 cột) là xác suất x thuộc class k đó

- Tạo 1 encoder và dùng public method `toarray()` của lớp này để mã hóa (fit_transform) `y_train` trước theo ý tưởng trên

In [351]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
y_train_new = enc.fit_transform(y_train.reshape(-1,1)) # fit_transform() expect vector cột nên phải reshape
y_train_new = y_train_new.toarray() # convert to np array để dùng

In [352]:
# test
print(y_train[:5])
y_train_new[:5]

[2 1 1 0 2]


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

- Giờ encode trên `y_val` và `y_test` (chỉ gọi `transform`)

In [353]:
y_val_new = enc.transform(y_val.reshape(-1,1)).toarray()
y_test_new = enc.transform(y_test.reshape(-1,1)).toarray()

# Tạo softmax function

- Với 1 instance x thì $s_k(x)$ gọi là logit, có kích thước 1x3 lưu score cho mỗi class mà x thuộc về 
- Tuy nhiên, cái ta tính là tính trên cả dataset (do dùng numpy) nên logit**s** là array của các logit đơn -> logit**s** có kích thước mx3
    - 1 con x (1x3, 3=2+1), 1 class k (3x1): $s_k(x) = x^T \theta$ = 1 số
    - 1 con x (1x3), k=3 class (3x3): `logit` = $s(x) = x^T \Theta = \begin{bmatrix} s_0(x) & s_1(x) & s_2(x) \end{bmatrix}$ = matrix kích thước 1x3 -> score cho mỗi class của 1 con x
    - m con x (mx3), k=3 class (3x3): `logits` = $\begin{bmatrix} s_0(x_0) & s_1(x_0) & s_2(x_0) \\ s_0(x_1) & s_1(x_1) & s_2(x_1) \\ ... \end{bmatrix}$ = matrix kích thước mx3 -> mỗi hàng là mỗi con $s(x_i)$

In [354]:
# p_hat
def softmax(logits):
    '''logits: shape = mxk, mỗi hàng thể hiện xác suất của 1 instance thuộc vào k class'''
    nume = np.exp(logits) # m x 3
    deno = np.sum(nume, axis=1, keepdims=True) # cộng theo hàng -> m x 1
    return nume/deno # m x 3

# Training

- Khởi tạo Theta $\Theta$ (theta lớn) có shape = (n+1, K) với mỗi cột là 1 bộ tham số $\theta _i$ của mỗi class; ứng với mỗi cột thì từng hàng là feature_weight với $w_0$ = 1 

In [355]:
n_inputs = X_train.shape[1] # == 3 (2 features plus the bias term)
n_outputs = len(np.unique(y_train))   # == 3 (3 iris classes)

def init_Theta():
    Theta = np.random.randn(n_inputs, n_outputs)
    return Theta

- Sử dụng code của thuật toán Batch GD để training tìm ra bộ tham số mô hình $\Theta$ tốt nhất. Trong quá đó lồng ghép 2 CT chính sau đây để giải:
1. The cost function: $J(\Theta) = - \frac 1 m \sum \limits _{i=1} ^{m} \sum \limits _{k=1} ^{K} y_k^{(i)} log(\hat{p}_k^{(i)})$ = $mean(sum(y_k^{(i)} log(\hat{p}_k^{(i)}), axis=1))$ = mean(sum(`y_train_new`*log(`y_proba`))) 
    - trong numpy, để tránh log(0) = nan, ta cộng 1 lượng `eps` vào log
2. gradient - đạo hàm từng phần: $\nabla _{\Theta}J(\Theta) = \frac 1 m XE^T$ với $E = \hat{P} - y\_train\_new$
- Thấy rằng cả 2 CT trên đều dùng $\hat{p}_k^{(i)}$ = `y_proba` là kq của softmax function nên tính $\hat{p}_k^{(i)}$ trước

In [356]:
learning_rate = 0.05
n_iterations = 5001
m = len(X_train)
eps = 1e-7

Theta = init_Theta()

for i in range(n_iterations):
    # tính P_hat = y_proba
    logits = X_train.dot(Theta) # shape = (m,3)
    y_proba = softmax(logits) # shape = (m,3)
    
    # tính loss function và in ra sau mỗi 500 vòng lặp, ta kỳ vọng loss luôn giảm sau mỗi iter
    if i % 500 == 0:
        loss = -np.mean(np.sum(y_train_new * np.log(y_proba + eps), axis=1)) # tích có shape = (m,3)
        print(i, loss)

    # tính hiệu trong hàm gradient: error
    E = y_proba - y_train_new # (m,3) - (m,1) = (m,3)
    gradient = 1/m * X_train.T.dot(E) # (n+1,3) = (3,3)
    Theta = Theta - learning_rate*gradient

0 7.671746502962418
500 0.513541064240001
1000 0.4008764512320962
1500 0.34751149803877507
2000 0.31369883147140937
2500 0.2893722811067073
3000 0.2706102255796254
3500 0.25550070017188903
4000 0.24296973909342823
4500 0.232352702981127
5000 0.22320874349690198


**Giải thích shape của 1 số phép toán**:
- `logits`.shape = (m,3): mỗi hàng (i.e. 1 instance) chứa 3 score cho 3 lớp -> broadcast cho m điểm dữ liệu tương ứng m hàng (lưu ý score có thể âm or dương)
- `y_proba`.shape = `logits`.shape = (m,3): mỗi hàng (i.e. 1 instance) chứa 3 probability cho 3 lớp -> broadcast cho m điểm dữ liệu tương ứng m hàng
- `gradient`.shape = (n+1,3) = (3,3): mỗi hàng là mỗi trọng số ($w_0$ - bias, $w_1$, $w_2$), mỗi cột là mỗi theta (i.e. mỗi class). Lý do có shape này đó là, để đạo hàm trên hàm TQ $J(\Theta)$, lấy đạo hàm từng phần trên mỗi $\theta _i$; mà mỗi $\theta _i$ lại có n+1=2+1=3 trọng số, như vậy lại phải lấy đạo hàm từng phần của $\theta _i$ trên 3 trọng số. Kết quả là `gradient` 
    - Nhớ lại lý thuyết, mỗi vector cột trong `gradient` là vector luôn đi lên dốc (uphill direction) nên lấy từng cột $\theta$ trong $\Theta$ trừ đi `learning_rate`*từng cột trong `gradient` sẽ update dc $\Theta$ (coi lại Batch GD)

In [357]:
Theta

array([[ 6.86782215,  0.54140246, -6.40308622],
       [-1.09227558,  1.0119759 ,  1.13503921],
       [-1.5704224 , -0.2863346 ,  3.73393016]])

# Prediction + Evaluation

- Giờ ta dự đoán trên tập valid để đánh giá performace. Tập valid có m' dòng, 3 cột

In [358]:
logits = X_val.dot(Theta) # shape = (m',3) 
y_proba = softmax(logits) # shape = (m',3)
y_val_pred = np.argmax(y_proba, axis=1)

# accuracy
val_acc = np.mean(y_val_pred == y_val)
val_acc

0.9166666666666666

Nhận xét: bằng việc toggle biến `n_iterations` và `learning_rate` thì hên nó ra cao, VD: val_acc = 91.66%

# Train model có sử dụng regularization

- Trong các hàm của scikit-learn thì default bên dưới có sử dụng regularization nên ta sẽ lập trình lại phần này luôn và ta chọn *Ridge regularization* (tức regularization term = l2)
- Như vậy từ đoạn code cũ cần chỉnh:
    - Định nghĩa lại hàm loss (i.e. đề bài): $J(\Theta) = J(\Theta) + \alpha * regularization\_term$ với `regularization_term` = $\alpha * \frac 1 2 \sum \limits _{i=1} ^n w_i^2 = l_2$
    - Do sử dụng Batch GD nên `gradient` = `gradient` + $\alpha$.**w**, vơi **w** = [$w_1$, $w_2$, ..., $w_n$]$^T$ (ko regularize $w_0$)

In [359]:
learning_rate = 0.1 # tăng
n_iterations = 7001
m = len(X_train)
eps = 1e-7
alpha = 0.1 # hyperparameter

Theta = init_Theta()

for i in range(n_iterations):
    # tính P_hat = y_proba
    logits = X_train.dot(Theta) # shape = (m,3)
    y_proba = softmax(logits) # shape = (m,3)
    
    # tính loss function và in ra sau mỗi 500 vòng lặp, ta kỳ vọng loss luôn giảm sau mỗi iter
    if i % 500 == 0:
        loss = -np.mean(np.sum(y_train_new * np.log(y_proba + eps), axis=1)) # tích có shape = (m,3)
        l2 = 1/2 * np.sum(np.square(Theta[1:])) # sum theo từng theta (hay axis=0) trong Theta 
        loss = loss + alpha * l2
        print(i, loss)

    # tính hiệu trong hàm gradient: error
    E = y_proba - y_train_new # (m,3) - (m,1) = (m,3)
    gradient = 1/m * X_train.T.dot(E) + alpha * np.r_[np.zeros([1, n_outputs]), Theta[1:]] # (n+1, 3)
    Theta = Theta - learning_rate * gradient

0 7.351186178750213
500 0.5363074274879401
1000 0.5103361469887799
1500 0.5028300175534949
2000 0.5000930424932675
2500 0.49900726252113436
3000 0.49855778929510985
3500 0.4983671456117274
4000 0.49828508447873926
4500 0.49824943422167567
5000 0.4982338546809579
5500 0.498227020110083
6000 0.4982240143362766
6500 0.4982226902490736
7000 0.4982221063362394


**Giải thích shape của 1 số phép toán**:
- `gradient`.shape = (n+1,3), shape ko đổi so với code cũ:  
    - `1/m * X_train.T.dot(E)` -> shape = (n+1,3): chính là gradient cũ, coi lại gthich ở các cell trên
    - `np.r_[np.zeros([1, n_outputs]), Theta[1:]]` = **w** = vector cột -> shape = (n+1,3): do ta ko regularize $w_0$ nên hàng đầu của  **w** chứa số 0 (tức `np.zeros([1, n_outputs])`), còn các hàng còn lại chứ [$w_1$, $w_2$, ..., $w_n$]$^T$ (tức `Theta[1:]`)
    - xong lấy tổng thì shape vẫn là (n+1,3), có điều hàng $0^{th}$ ko thay đổi sau khi cộng (thỏa mãn "ko regularize $w_0$")

In [360]:
logits = X_val.dot(Theta)
y_proba = softmax(logits)
y_val_pred = np.argmax(y_proba, axis=1)

# accuracy
np.mean(y_val_pred == y_val)

0.9166666666666666

# Training có sử dụng regularization + early stopping

Note: ta chỉ cập nhật Theta trên X_train, X_val dc thêm vào chỉ mang t/c lợi dùng MSE_val của nó để làm DK dừng lặp

In [391]:
learning_rate = 0.1 # tăng
n_iterations = 7001
m = len(X_train)
eps = 1e-7
alpha = 0.1 # hyperparameter
best_loss = np.inf

Theta = init_Theta()

for i in range(n_iterations):
    # tính trên X_train
    logits = X_train.dot(Theta) # shape = (m,3)
    y_proba = softmax(logits) # shape = (m,3)
    E = y_proba - y_train_new # (m,3) - (m,1) = (m,3)
    gradient = 1/m * X_train.T.dot(E) + alpha * np.r_[np.zeros([1, n_outputs]), Theta[1:]] # (n+1, 3)
    Theta = Theta - learning_rate * gradient
    
    # tính trên X_val
    logits = X_val.dot(Theta) # shape = (m,3)
    y_proba = softmax(logits) # shape = (m,3)
    
    ## tính MSE_val
    loss = -np.mean(np.sum(y_val_new * np.log(y_proba + eps), axis=1)) # tích có shape = (m,3)
    l2 = 1/2 * np.sum(np.square(Theta[1:])) # sum theo từng theta (hay axis=0) trong Theta 
    loss = loss + alpha * l2
    if i % 500 == 0:
        print(i, loss)
    if loss < best_loss:
        best_loss = loss
    else: # DK dừng: ngay khi MSE_val tăng thì dừng gấp
        print(i - 1, best_loss) # show best_loss
        print(i, loss, "early stopping!") # show loss ngay khi lớn best_loss 
        break

0 2.584472127756714
500 0.6083720338049515
1000 0.5618544074845901
1500 0.5472862318780056
2000 0.5411347650851447
2500 0.5381617580416181
3000 0.5365935456005979
3500 0.5357118359742908
4000 0.5351918062819365
4500 0.5348739945642981
5000 0.5346746804486506
5500 0.5345473625599602
6000 0.5344649844756109
6500 0.5344112113320107
7000 0.5343758989861248


In [392]:
logits = X_val.dot(Theta)
y_proba = softmax(logits)
y_val_pred = np.argmax(y_proba, axis=1)

# accuracy
np.mean(y_val_pred == y_val)

0.9166666666666666

Nhận xét: tập validation có dùng early stopping dự đoán đúng 91.67%. Bây giờ dự đoán trên tập test

In [393]:
logits = X_test.dot(Theta)
y_proba = softmax(logits)
y_test_pred = np.argmax(y_proba, axis=1)

accuracy_score = np.mean(y_test_pred == y_test)
accuracy_score

1.0

Nhận xét: dự đoán đúng 100%, có lẽ do tập test nhỏ mới đạt dc độ hoàn hảo như vầy..